In [2]:
from statsmodels.tsa.stattools import adfuller 
from numpy import log 
import numpy as np
import pandas as pd 
from scipy.optimize import least_squares

In [36]:
dQI = pd.read_csv('dQI.csv', index_col=False) 

/tmp/ipykernel_22616/3907934338.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  dQI = pd.read_csv('dQI.csv', index_col=False)


In [37]:
print(dQI)

        Time     Impact      DeltaQ
0    0.02008  59.303139  139.456975
1    0.02014  22.280227   30.506213
2    0.03036  41.082159  127.648119
3    0.03040   0.103186    4.639194
4    0.03066   0.212766    7.029081
..       ...        ...         ...
465  0.77008  34.208399  167.643592
466  0.79962   4.439648    6.466755
467  0.79970   4.439648    6.466755
468  0.93328  47.422737  222.611009
469  0.93842  29.270251  139.597557

[470 rows x 3 columns]


In [38]:
class OnlineData:

    def __init__(self, filename):
        self.filename = filename
        self.f = open(filename, 'r')

    def __next__(self):
        line = next(self.f)[:-1]
        if len(line) < 1:
            return None
        Q_, P_ = np.array([z[1:-1].split(",") for z in line.split("\t")
                          ]).astype(float).astype(int).T
        CS = np.cumsum(P_ * np.ediff1d(Q_, to_begin=Q_[0]))

        def F(Q):
            if Q <= Q_[0]:
                return P_[0] * Q
            if Q > Q_[-1]:
                return None
            i = np.searchsorted(Q_, Q)
            return CS[i - 1] + (Q - Q_[i - 1]) * P_[i]

        self.F = F
        return self.F

    def __iter__(self):
        return self

    def close(self):
        self.f.close()

0      0.02008
1      0.02014
2      0.03036
3      0.03040
4      0.03066
        ...   
465    0.77008
466    0.79962
467    0.79970
468    0.93328
469    0.93842
Name: Time, Length: 470, dtype: float64


In [1]:
from statsmodels.tsa.stattools import adfuller 
from numpy import log 
import numpy as np
import pandas as pd 
from scipy.optimize import least_squares

class GOWModel:
    '''
    The Implementation of the Generalized Obyzhaeva-Wang Market Impact model.
    We predict MI in the form I(t_{i+1}) = \rho^{t_{i+1} - t_{i}} I(t_{i}) + \lamda \Delta Q,
    where I is MI and \Delta Q is a change of our position.
    Details in A3 of 'Handbook of Market Impact Modelling'
    '''
    def __init__(self, lambd=1., rho=1.):
        self.lambd = lambd
        self.rho = rho

    def fitLS(self, # This method fits the model using least_squares from scipy.optimize
            MI: np.array, # Market Impact array 
            dQ: np.array,  # Changes of position array
            T: np.array # The array of moments of position changes 
           ):
        '''
        This method fits the model using least_squares from scipy.optimize
        '''
        def fun(x, mi: np.array, mi_prev: np.array, dq: np.array, dt: np.array): # residual function to optimize - see the scipy.optimize.least_squares docs
            return np.power(x[0], dt) * mi_prev + x[1] * dq - mi
        
        res_lsq = least_squares(fun, np.array([self.rho, self.lambd]), args=(MI[1:], MI[:-1], dQ[:-1], np.diff(T)))

        self.rho = res_lsq.x[0]
        self.lambd = res_lsq.x[1]
        self.result = res_lsq
        self.lastMI = MI[-1] 

    def predict(self, 
                dQ: np.array, # Changes of position array
                T: np.array, # The array of moments of position changes
                MI: np.array
                ):
        return np.power(self.rho, np.diff(T)) * MI[:-1] + self.lambd * dQ[1:]
    
    def predict(self, 
                dQ: np.array, # Changes of position array
                T: np.array, # The array of moments of position changes
                ):
        ans = np.array([])
        for i in range(len(np.diff(T))):
            pmi = np.power(self.rho, np.diff(T)[i]) * self.lastMI + self.lambd * dQ[i]
            np.append(ans, pmi)
            self.lastMI = pmi

        return ans
        

In [3]:
dQI = pd.read_csv('DataForImpact/dQI.csv', index_col=False) 

FileNotFoundError: [Errno 2] No such file or directory: 'DataForImpact/dQI.csv'